In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 

In [2]:
data_url='http://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data'
names = ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'COMFORT', 'decision']
df = pd.read_csv(data_url, names=names)
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [3]:
df['decision'][3]=df['decision'][4]
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [4]:
#COMPORT 데이터 전처리 '?'값 평균값으로 처리

int_data = df['COMFORT'] [df['COMFORT'] != '?'].astype('int64')

average=sum(int_data)/len(int_data)

df['COMFORT'] [df['COMFORT'] == '?']=str(average)

for c in ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'decision','COMFORT']:
    df[c] = df[c].astype('category')

df["COMFORT"]=df["COMFORT"].astype(np.float32)

In [5]:
y_data=df.pop('decision')
x_data=pd.get_dummies(df).values


In [6]:
df=pd.get_dummies(df)

In [7]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_data[:,0]=min_max_scaler.fit_transform(x_data[:,0])

x_data[0,:]

/home/yoonjin/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/yoonjin/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


array([ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,
        0.,  0.,  1.,  0.,  0.,  1.,  0.])

In [ ]:
#LogisticRegression 모델 사용 후 성능 평가

from sklearn.cross_validation import KFold,ShuffleSplit
from sklearn.linear_model import LogisticRegression

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))

In [ ]:
# x 데이터에 Y의 S 라벨이 가장 영향을 많이 받으면서, L-BP_low와 CORE_STBL_unstable 컬럼이 가장 상관도가 높음
#COMFORT 변수는 drop시길 예정

y_dummies=pd.get_dummies(y_data)

print('---------------S----------------')
print(df.corrwith(y_dummies["S"]).sort_values(ascending=True))
print('---------------A----------------')
print(df.corrwith(y_dummies["A"]).sort_values(ascending=True))
print('---------------I----------------')
print(df.corrwith(y_dummies["I"]).sort_values(ascending=True))

In [ ]:
#feature engineering (추가 & 삭제)
import copy

df2=df.copy()

#y와 상관도가 높은 변수 생성
df2['HIGH_REL'] = df['CORE-STBL_unstable']*100000+df['L-BP_low']*10000000

df2=df2.drop(["COMFORT"],axis=1)
df2=df2.values

In [ ]:
x_data=df2

In [ ]:
#feature engineering 후 재평가

from sklearn.cross_validation import KFold,ShuffleSplit
from sklearn.naive_bayes import MultinomialNB

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))

In [ ]:
#decsion tree 모델 사용 후 성능 평가

from sklearn import tree

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(x_data, y_data)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))
